# 🧠 Assistant OS Factory v1.0
This notebook transforms raw assistant markdowns into GPT-enhanced, Streamlit-ready mini-apps with ZIP bundling and metadata tracking.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from google.colab import userdata
import os

# 🔐 Load OpenAI key from Colab Secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI")


In [ ]:
import os

INPUT_DIR = "/content/drive/MyDrive/assistant_markdown/raw/"
OUTPUT_DIR = "/content/drive/MyDrive/assistant_markdown/processed/"
LAUNCHER_DIR = "/content/drive/MyDrive/assistant_markdown/streamlit_ready/"
ZIP_EXPORT_PATH = "/content/drive/MyDrive/assistant_markdown/assistant_bundle.zip"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LAUNCHER_DIR, exist_ok=True)


In [ ]:
!pip install openai -q

import openai
import json
import zipfile
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
def load_md_files(directory):
    return [f for f in os.listdir(directory) if f.endswith(".md")]


In [ ]:
def enhance_markdown(md_text):
    prompt = f"""You are an AI assistant markdown enhancer. Given the raw markdown below, clean and complete it by:
- Filling in missing fields (e.g., description, input/output, category)
- Ensuring clarity and correct formatting
- Suggesting a better title if needed

Return only the improved markdown.

### Raw Markdown:
{md_text}
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


In [ ]:
import re

def parse_markdown(md_text):
    fields = {
        "title": re.search(r"^# (.+)", md_text, re.MULTILINE),
        "description": re.search(r"## Description\n([\s\S]+?)\n(?:##|Category:|- Input:|- Output:|$)", md_text),
        "category": re.search(r"Category: (.+)", md_text),
        "inputs": re.findall(r"- Input: (.+)", md_text),
        "outputs": re.findall(r"- Output: (.+)", md_text),
    }
    return {k: (v.group(1).strip() if v else None) if not isinstance(v, list) else v for k, v in fields.items()}


In [ ]:
def generate_streamlit_files(data, enhanced_md, output_folder):
    name = data.get('title', 'untitled').lower().replace(" ", "_")
    folder = os.path.join(output_folder, name)
    os.makedirs(folder, exist_ok=True)

    with open(os.path.join(folder, f"{name}.md"), "w") as f:
        f.write(enhanced_md)

    with open(os.path.join(folder, f"{name}.py"), "w") as f:
        f.write(f"""import streamlit as st

def run_ui():
    st.title("{data.get('title', 'Untitled')}")
    st.write("{data.get('description', 'No description provided.')}")
    # Add UI logic here

if __name__ == '__main__':
    run_ui()
""")
    manifest = {
        "title": data.get('title', 'Untitled'),
        "category": data.get("category", "Uncategorized"),
        "uses_gpt": "gpt" in enhanced_md.lower(),
        "has_run_ui": True,
        "path": folder
    }
    with open(os.path.join(folder, "manifest.json"), "w") as f:
        json.dump(manifest, f, indent=2)

    with open(os.path.join(folder, "README.md"), "w") as f:
        f.write(f"""# {data.get('title', 'Untitled')}

## Description
{data.get('description', 'N/A')}

### Category
{data.get('category', 'Uncategorized')}

### Inputs
- {'\n- '.join(data.get('inputs', []))}

### Outputs
- {'\n- '.join(data.get('outputs', []))}
""")
    return folder


In [ ]:
summary_manifest = []
md_files = load_md_files(INPUT_DIR)
for filename in md_files:
    with open(os.path.join(INPUT_DIR, filename), "r") as f:
        raw_md = f.read()

    enhanced_md = enhance_markdown(raw_md)
    parsed_data = parse_markdown(enhanced_md)

    new_name = filename.replace(".md", "_enhanced.md")
    with open(os.path.join(OUTPUT_DIR, new_name), "w") as f:
        f.write(enhanced_md)

    assistant_folder = generate_streamlit_files(parsed_data, enhanced_md, LAUNCHER_DIR)
    summary_manifest.append({
        "filename": filename,
        "title": parsed_data.get("title"),
        "path": assistant_folder
    })
    print(f"✅ Assistant ready: {filename} → {assistant_folder}")


In [ ]:
with open(os.path.join(LAUNCHER_DIR, "zip_manifest.json"), "w") as f:
    json.dump(summary_manifest, f, indent=2)


In [ ]:
def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                full_path = os.path.join(root, file)
                arcname = os.path.relpath(full_path, folder_path)
                zipf.write(full_path, arcname)
    return zip_path

zip_folder(LAUNCHER_DIR, ZIP_EXPORT_PATH)
print(f"📦 Bundle saved at: {ZIP_EXPORT_PATH}")


In [ ]:
# ✅ Enhanced: Launcher with Validation & GPT Suggestions

def generate_smart_launcher(launcher_dir, output_path):
    import os

    launcher_code = '''import streamlit as st
import importlib.util
import os
import json
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

st.set_page_config(page_title="Smart Assistant Launcher", layout="wide")
st.title("🧠 Modular Assistant Launcher")

ASSISTANT_DIR = os.path.dirname(__file__)
tabs = []
modules = []
summaries = []

# Discover assistant modules
for folder in sorted(os.listdir(ASSISTANT_DIR)):
    folder_path = os.path.join(ASSISTANT_DIR, folder)
    if os.path.isdir(folder_path):
        manifest_path = os.path.join(folder_path, "manifest.json")
        py_files = [f for f in os.listdir(folder_path) if f.endswith(".py")]

        summary = {"title": folder.replace("_", " ").title(), "gpt": False, "ui": False, "folder": folder}
        if os.path.exists(manifest_path):
            with open(manifest_path) as f:
                data = json.load(f)
                summary["title"] = data.get("title", summary["title"])
                summary["gpt"] = data.get("uses_gpt", False)
                summary["ui"] = data.get("has_run_ui", False)

        for py_file in py_files:
            full_path = os.path.join(folder_path, py_file)
            spec = importlib.util.spec_from_file_location(folder, full_path)
            module = importlib.util.module_from_spec(spec)
            try:
                spec.loader.exec_module(module)
                if hasattr(module, "run_ui"):
                    tabs.append(summary["title"])
                    modules.append(module)
                    summaries.append(summary)
            except Exception as e:
                print(f"Error loading {folder}: {e}")

if tabs:
    selected = st.selectbox("Choose an Assistant", tabs)
    selected_index = tabs.index(selected)
    st.divider()
    
    st.markdown(f"**Folder:** `{summaries[selected_index]['folder']}`")
    st.markdown("### ✅ Validation")
    col1, col2, col3 = st.columns(3)
    col1.success("Has `run_ui()`") if summaries[selected_index]["ui"] else col1.error("Missing `run_ui()`")
    col2.info("Uses GPT") if summaries[selected_index]["gpt"] else col2.warning("No GPT reference")
    col3.success("Manifest Found")

    # GPT Suggestion Area
    folder_path = os.path.join(ASSISTANT_DIR, summaries[selected_index]["folder"])
    md_file = next((f for f in os.listdir(folder_path) if f.endswith(".md")), None)

    if md_file:
        st.divider()
        st.markdown("### 💡 GPT Suggestions")
        with open(os.path.join(folder_path, md_file)) as f:
            md_content = f.read()

        if st.button("🔮 Suggest Next Assistant"):
            try:
                suggestion_prompt = f"Based on this assistant, suggest a useful next assistant idea:

{md_content}"
                response = openai.ChatCompletion.create(
                    model="gpt-4",
                    messages=[{"role": "user", "content": suggestion_prompt}],
                    temperature=0.5
                )
                st.success(response.choices[0].message.content.strip())
            except Exception as e:
                st.error(f"GPT error: {e}")
else:
    st.info("No assistants with `run_ui()` found.")
'''
    with open(output_path, "w") as f:
        f.write(launcher_code)

generate_smart_launcher(LAUNCHER_DIR, os.path.join(LAUNCHER_DIR, "multi_tab_launcher.py"))
print("✅ Smart Streamlit launcher with validation + GPT tips created.")


🎉 **DONE:** Your Assistant OS Factory is complete and ready to deploy!